#//*[@id="main"]/div[2]/div[9]

In [11]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
url = "https://pvpoketw.com/rankings/all/1500/leads/"
driver.get(url)
driver.minimize_window()
# 等待網頁加載完成
time.sleep(5)
table = driver.find_element(By.XPATH, '//*[@id="main"]/div[2]/div[9]')
text = table.text
text = text.replace('\n†\n,', ',')
text = text.replace('\n†\n', '\n')
text = text.split('\n')


In [12]:
from flask import Flask, render_template
app = Flask(__name__)

table_rows = []
@app.route('/')
def display_table():

    
    # 每三項為一組，以並排顯示
    for i in range(0, len(text), 3):
        name = text[i]
        moves = text[i + 1]
        
        score = text[i + 2]
        
        
        table_rows.append({
            'name': name,
            'moves': moves,
            'score': score
        })

    return render_template('index.html', rows=table_rows)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Jun/2023 21:48:53] "GET / HTTP/1.1" 200 -


In [13]:
import pyodbc

# 建立與 SQL Server 的連線
conn = pyodbc.connect("Driver={SQL Server};SERVER=MSI;DATABASE=poke;UID=sa;PWD=123")

# 確認連線成功
if conn:
    print("OK")

# 建立 cursor 來執行 SQL 語法
cursor = conn.cursor()




OK


In [ ]:
cursor.execute('''
		CREATE TABLE cp1500 (
            name nvarchar(50),
            moves nvarchar(150),
            score nvarchar(50)
			)
               ''')

conn.commit()

In [ ]:
#更換moves欄位的逗號為底線
for row in table_rows:
    moves = row['moves'].replace(',', '_')
    row['moves'] = moves

In [ ]:
#插入資料
insert_sql = "INSERT INTO cp1500 (name, moves, score) VALUES (?, ?, ?)"

for row in table_rows:
    values = (row['name'], row['moves'], row['score'])
    cursor.execute(insert_sql, values)

# 提交資料庫交易
conn.commit()


In [14]:
cursor.execute('SELECT * FROM cp1500')

# 獲取查詢結果
result = cursor.fetchall()

# 輸出結果
for row in result:
    print(row)

# 關閉連線
cursor.close()
conn.close()


('#1大嘴鷗', '翅膀攻擊_ 氣象球 水_ 暴風', '100')
('#2頭巾混混', '雙倍奉還_ 欺詐_ 增強拳', '99.6')
('#3巨沼怪 暗影', '泥巴射擊_ 加農水砲*_ 地震', '99')
('#4恰雷姆XL', '雙倍奉還_ 冰凍拳_ 精神強念', '98.9')
('#5大狃拉', '暗影爪_ 近身戰_ 十字剪', '98.7')
('#6巨沼怪', '泥巴射擊_ 加農水砲*_ 地震', '98.7')
('#7倫琴貓 暗影', '電光_ 瘋狂伏特_ 精神之牙*', '98.6')
('#8沼王 暗影', '泥巴射擊_ 泥巴炸彈_ 尖石攻擊', '98.4')
('#9怪力 暗影', '雙倍奉還_ 十字劈_ 岩崩', '98.1')
('#10大比鳥', '翅膀攻擊*_ 羽毛舞_ 勇鳥猛攻', '97.4')
('#11泥巴魚 伽勒爾', '泥巴射擊_ 岩崩_ 地震', '96.9')
('#12天蠍王', '翅膀攻擊_ 地震_ 暗襲要害', '96.6')
('#13怪力', '雙倍奉還_ 十字劈_ 岩崩', '96.6')
('#14噴火龍 暗影', '翅膀攻擊*_ 爆炸烈焰*_ 龍爪', '96.4')
('#15倫琴貓', '電光_ 瘋狂伏特_ 精神之牙*', '96.1')
('#16托戈德瑪爾', '電擊_ 致命針刺_ 瘋狂伏特', '96')
('#17過動猿', '雙倍奉還_ 泰山壓頂_ 重踏', '96')
('#18杖尾鱗甲龍', '龍尾_ 近身戰_ 龍爪', '95.7')
('#19火爆猴', '雙倍奉還_ 暗襲要害_ 近身戰', '95.4')
('#20電燈怪', '電光_ 衝浪_ 十萬伏特', '95.1')
('#21巨翅飛魚', '翅膀攻擊_ 泡沫光線_ 冰凍光束', '95.1')
('#22貓頭夜鷹', '翅膀攻擊_ 神鳥猛擊_ 暗影球', '95.1')
('#23胖嘟嘟', '禍不單行_ 衝浪_ 暗影球', '94.9')
('#24雷吉斯奇魯', '鎖定_ 真氣彈_ 電磁炮*', '94.9')
('#25噴火龍', '翅膀攻擊*_ 爆炸烈焰*_ 龍爪', '94.8')
('#26大舌頭XL', '舌舔_ 泰山壓頂*_ 強力鞭打', '94.8')
('#27穿山王 阿羅拉 暗影', '